In [1]:
import base64
import json
import random

import pandas as pd
import requests
import tensorflow as tf

In [3]:
df = pd.read_csv("data/heart.csv")
y = df.pop("target")

columns = df.columns.values
rand = random.randint(0, len(df))

features = df.values[rand]
label = y[rand]

inputs = {key: value for key, value in zip(columns, features)}
inputs

{'age': 46.0,
 'sex': 0.0,
 'chest_pain_type': 1.0,
 'resting_bp': 105.0,
 'cholestoral': 204.0,
 'fasting_blood_sugar': 0.0,
 'restecg': 1.0,
 'max_hr': 172.0,
 'exang': 0.0,
 'oldpeak': 0.0,
 'slope': 2.0,
 'num_major_vessels': 0.0,
 'thal': 2.0}

In [4]:
def string_feature(value):
    return tf.train.Feature(
        bytes_list=tf.train.BytesList(value=[bytes(value, "utf-8")]),
    )


def float_feature(value):
    return tf.train.Feature(
        float_list=tf.train.FloatList(value=[value]),
    )


def int_feature(value):
    return tf.train.Feature(
        int64_list=tf.train.Int64List(value=[value]),
    )

In [18]:
def prepare_json(inputs: dict):
    feature_spec = dict()

    for keys, values in inputs.items():
        if keys == "oldpeak":
            feature_spec[keys] = float_feature(float(values))
        else:
            feature_spec[keys] = int_feature(int(values))

    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()

    result = [{"examples": {"b64": base64.b64encode(example).decode()}}]

    return json.dumps(
        {
            "signature_name": "serving_default",
            "instances": result,
        }
    )

In [20]:
def make_predictions(inputs):
    json_data = prepare_json(inputs)

    endpoint = (
        "https://heart-disease-prediction-production-a4aa.up.railway.app/v1/models/heart-disease-model:predict"
    )
    response = requests.post(endpoint, data=json_data)
    
    # Print the response status and content for debugging
    print("Response Status Code:", response.status_code)
    print("Response Content:", response.text)

    # Attempt to parse the prediction
    try:
        prediction = response.json()["predictions"][0][0]
    except KeyError:
        return "Error: 'predictions' key not found in the response."

    if prediction < 0.5:
        return "No disease"
    else:
        return "Disease"


In [21]:
make_predictions(inputs)

Response Status Code: 200
Response Content: {
    "predictions": [[0.993837714]
    ]
}


'Disease'

In [22]:
columns = df.columns.values
rand = random.randint(0, len(df))

features = df.values[rand]
label = y[rand]

inputs = {key: value for key, value in zip(columns, features)}
inputs

{'age': 60.0,
 'sex': 1.0,
 'chest_pain_type': 2.0,
 'resting_bp': 140.0,
 'cholestoral': 185.0,
 'fasting_blood_sugar': 0.0,
 'restecg': 0.0,
 'max_hr': 155.0,
 'exang': 0.0,
 'oldpeak': 3.0,
 'slope': 1.0,
 'num_major_vessels': 0.0,
 'thal': 2.0}

In [23]:
make_predictions(inputs)

Response Status Code: 200
Response Content: {
    "predictions": [[0.917447329]
    ]
}


'Disease'

In [24]:
columns = df.columns.values
rand = random.randint(0, len(df))

features = df.values[rand]
label = y[rand]

inputs = {key: value for key, value in zip(columns, features)}
inputs

{'age': 44.0,
 'sex': 1.0,
 'chest_pain_type': 0.0,
 'resting_bp': 112.0,
 'cholestoral': 290.0,
 'fasting_blood_sugar': 0.0,
 'restecg': 0.0,
 'max_hr': 153.0,
 'exang': 0.0,
 'oldpeak': 0.0,
 'slope': 2.0,
 'num_major_vessels': 1.0,
 'thal': 2.0}

In [25]:
make_predictions(inputs)

Response Status Code: 200
Response Content: {
    "predictions": [[0.0458697714]
    ]
}


'No disease'